# python对excel数据清洗合并实践案例

## 简要说明：
数据是一个地震灾后各个家户收到的救助金与他们的家庭经济情况相关的数据。  
数据文件data1,data2  
1.对EE17AB-EE17EB金额加总，计算出一个救助金总额变量（在data2中）  
2.EE01、EE02：可以区分该家户在震前是否接受政府救济（在data2中）（删除数字8）
3.EE03：家庭经济状况5个分类的自评得分（在data2中） （删除数字8）  
4.LF07：地震前家庭中每个人的月收入加总为家庭月收入（data1中是每个家庭成员具体的收入情况，需要对其进行汇总，汇总成每个家庭的收入情况）（data1）  （删除数字8）
对数据进行清晰，合并。  

## 具体步骤

### 用data1的数据，分类汇总，计算出每个家庭的总收入

In [93]:
#导入包
import pandas as pd 
import numpy as np
import xlrd

In [94]:
#导入数据data1,data2
df1 = pd.read_excel(r"D:/闲鱼买家/买家201-作业题目/data1.xlsx") #引号中是文件的完整路径，注意前面要写r。
df2 = pd.read_excel(r"D:/闲鱼买家/买家201-作业题目/data2.xlsx")

### *除去不需要的列：由于data1,data2中不是每一列数据都需要，因此，把需要的列提出来单独形成一个新的dataframe

In [95]:
df1 = df1[["ID","LF07"]] #i提取出ID,LF07两列
df1  #结果如下所示

,ID,LF07
0,1,0.0
1,1,0.0
2,1,1000.0
3,1,1500.0
4,1,NaN
...,...,...
12702,6265,0.0
12703,6265,0.0
12704,6265,0.0
12705,6265,1000.0


### 由于data1中给的是每个成员的收入，而非一个家庭的收入，所以还需要对其进行进一步处理，进行分类汇总。

In [96]:
df1=df1.groupby("ID").sum()#groupby函数实现分类汇总，还有其他方法，可以参考https://zhuanlan.zhihu.com/p/49945514
df1

,LF07
ID,
1,2500.0
2,1000.0
4,2000.0
5,0.0
6,2700.0
...,...
6261,1000.0
6262,0.0
6263,1000.0


In [97]:
df2 = df2[["ID","EE01","EE02","EE03","EE17AB","EE17BB","EE17CB","EE17DB","EE17EB"]]
df2

,ID,EE01,EE02,EE03,EE17AB,EE17BB,EE17CB,EE17DB,EE17EB
0,1,2.0,2.0,3.0,NaN,4.0,NaN,NaN,NaN
1,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11,2.0,2.0,2.0,NaN,500.0,NaN,NaN,NaN
3,13,1.0,2.0,3.0,NaN,190.0,NaN,NaN,NaN
4,14,2.0,2.0,3.0,NaN,400.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2230,6250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2231,6252,2.0,2.0,3.0,NaN,900.0,2000.0,NaN,NaN
2232,6253,2.0,2.0,3.0,NaN,1200.0,2000.0,NaN,NaN
2233,6256,2.0,2.0,3.0,NaN,900.0,2000.0,NaN,NaN


### 对比匹配，从上面我们可以看到，data1与data2数据的ID不是完全相同的，因此我们要找到两者共有的ID，然后匹配对应的信息。

实现Excel里面类似vlookup函数的功能

In [98]:
df3 = pd.merge(df1,df2,on="ID",how="inner")
df3

,ID,LF07,EE01,EE02,EE03,EE17AB,EE17BB,EE17CB,EE17DB,EE17EB
0,1,2500.0,2.0,2.0,3.0,NaN,4.0,NaN,NaN,NaN
1,11,4300.0,2.0,2.0,2.0,NaN,500.0,NaN,NaN,NaN
2,13,1700.0,1.0,2.0,3.0,NaN,190.0,NaN,NaN,NaN
3,14,2800.0,2.0,2.0,3.0,NaN,400.0,NaN,NaN,NaN
4,15,300.0,2.0,2.0,4.0,NaN,90.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1792,6246,1500.0,2.0,2.0,3.0,NaN,1200.0,2000.0,NaN,NaN
1793,6252,1000.0,2.0,2.0,3.0,NaN,900.0,2000.0,NaN,NaN
1794,6253,600.0,2.0,2.0,3.0,NaN,1200.0,2000.0,NaN,NaN
1795,6256,1000.0,2.0,2.0,3.0,NaN,900.0,2000.0,NaN,NaN


In [101]:
df3["incomeall"]=df3["EE17AB"]+df3["EE17BB"]
df3

,ID,LF07,EE01,EE02,EE03,EE17AB,EE17BB,EE17CB,EE17DB,EE17EB,incomeall
0,1,2500.0,2.0,2.0,3.0,NaN,4.0,NaN,NaN,NaN,NaN
1,11,4300.0,2.0,2.0,2.0,NaN,500.0,NaN,NaN,NaN,NaN
2,13,1700.0,1.0,2.0,3.0,NaN,190.0,NaN,NaN,NaN,NaN
3,14,2800.0,2.0,2.0,3.0,NaN,400.0,NaN,NaN,NaN,NaN
4,15,300.0,2.0,2.0,4.0,NaN,90.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1792,6246,1500.0,2.0,2.0,3.0,NaN,1200.0,2000.0,NaN,NaN,NaN
1793,6252,1000.0,2.0,2.0,3.0,NaN,900.0,2000.0,NaN,NaN,NaN
1794,6253,600.0,2.0,2.0,3.0,NaN,1200.0,2000.0,NaN,NaN,NaN
1795,6256,1000.0,2.0,2.0,3.0,NaN,900.0,2000.0,NaN,NaN,NaN


In [102]:
df3["EE17AB"].fillna(0)

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
1792    0.0
1793    0.0
1794    0.0
1795    0.0
1796    0.0
Name: EE17AB, Length: 1797, dtype: float64

In [104]:
df3["EE17AB"]=df3["EE17AB"].fillna(0)

In [105]:
df3

,ID,LF07,EE01,EE02,EE03,EE17AB,EE17BB,EE17CB,EE17DB,EE17EB,incomeall
0,1,2500.0,2.0,2.0,3.0,0.0,4.0,NaN,NaN,NaN,NaN
1,11,4300.0,2.0,2.0,2.0,0.0,500.0,NaN,NaN,NaN,NaN
2,13,1700.0,1.0,2.0,3.0,0.0,190.0,NaN,NaN,NaN,NaN
3,14,2800.0,2.0,2.0,3.0,0.0,400.0,NaN,NaN,NaN,NaN
4,15,300.0,2.0,2.0,4.0,0.0,90.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1792,6246,1500.0,2.0,2.0,3.0,0.0,1200.0,2000.0,NaN,NaN,NaN
1793,6252,1000.0,2.0,2.0,3.0,0.0,900.0,2000.0,NaN,NaN,NaN
1794,6253,600.0,2.0,2.0,3.0,0.0,1200.0,2000.0,NaN,NaN,NaN
1795,6256,1000.0,2.0,2.0,3.0,0.0,900.0,2000.0,NaN,NaN,NaN


In [106]:
df3[["EE17BB","EE17CB","EE17DB","EE17EB"]]=df3[["EE17BB","EE17CB","EE17DB","EE17EB"]].fillna(0)
df3

,ID,LF07,EE01,EE02,EE03,EE17AB,EE17BB,EE17CB,EE17DB,EE17EB,incomeall
0,1,2500.0,2.0,2.0,3.0,0.0,4.0,0.0,0.0,0.0,NaN
1,11,4300.0,2.0,2.0,2.0,0.0,500.0,0.0,0.0,0.0,NaN
2,13,1700.0,1.0,2.0,3.0,0.0,190.0,0.0,0.0,0.0,NaN
3,14,2800.0,2.0,2.0,3.0,0.0,400.0,0.0,0.0,0.0,NaN
4,15,300.0,2.0,2.0,4.0,0.0,90.0,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1792,6246,1500.0,2.0,2.0,3.0,0.0,1200.0,2000.0,0.0,0.0,NaN
1793,6252,1000.0,2.0,2.0,3.0,0.0,900.0,2000.0,0.0,0.0,NaN
1794,6253,600.0,2.0,2.0,3.0,0.0,1200.0,2000.0,0.0,0.0,NaN
1795,6256,1000.0,2.0,2.0,3.0,0.0,900.0,2000.0,0.0,0.0,NaN


In [109]:
df3["incomeall"]=df3["EE17AB"]+df3["EE17BB"]+df3["EE17CB"]+df3["EE17DB"]+df3["EE17EB"]
df3

,ID,LF07,EE01,EE02,EE03,EE17AB,EE17BB,EE17CB,EE17DB,EE17EB,incomeall
0,1,2500.0,2.0,2.0,3.0,0.0,4.0,0.0,0.0,0.0,4.0
1,11,4300.0,2.0,2.0,2.0,0.0,500.0,0.0,0.0,0.0,500.0
2,13,1700.0,1.0,2.0,3.0,0.0,190.0,0.0,0.0,0.0,190.0
3,14,2800.0,2.0,2.0,3.0,0.0,400.0,0.0,0.0,0.0,400.0
4,15,300.0,2.0,2.0,4.0,0.0,90.0,0.0,0.0,0.0,90.0
...,...,...,...,...,...,...,...,...,...,...,...
1792,6246,1500.0,2.0,2.0,3.0,0.0,1200.0,2000.0,0.0,0.0,3200.0
1793,6252,1000.0,2.0,2.0,3.0,0.0,900.0,2000.0,0.0,0.0,2900.0
1794,6253,600.0,2.0,2.0,3.0,0.0,1200.0,2000.0,0.0,0.0,3200.0
1795,6256,1000.0,2.0,2.0,3.0,0.0,900.0,2000.0,0.0,0.0,2900.0


现在我们只需要ID,LF07,EE01,EE02,EE03,incomeall这几个变量，其他的我们不需要了，因此，我们就把这几个提取出来

In [110]:
df4 = df3[["ID","LF07","EE01","EE02","EE03","incomeall"]]
df4

,ID,LF07,EE01,EE02,EE03,incomeall
0,1,2500.0,2.0,2.0,3.0,4.0
1,11,4300.0,2.0,2.0,2.0,500.0
2,13,1700.0,1.0,2.0,3.0,190.0
3,14,2800.0,2.0,2.0,3.0,400.0
4,15,300.0,2.0,2.0,4.0,90.0
...,...,...,...,...,...,...
1792,6246,1500.0,2.0,2.0,3.0,3200.0
1793,6252,1000.0,2.0,2.0,3.0,2900.0
1794,6253,600.0,2.0,2.0,3.0,3200.0
1795,6256,1000.0,2.0,2.0,3.0,2900.0


EE01.EE02,EE03中，值为8的删去

In [114]:
df4 = df4[~(df4['EE01'].isin([8])|df4['EE02'].isin([8])|df4['EE03'].isin([8]))]#~取反
df4

,ID,LF07,EE01,EE02,EE03,incomeall
0,1,2500.0,2.0,2.0,3.0,4.0
1,11,4300.0,2.0,2.0,2.0,500.0
2,13,1700.0,1.0,2.0,3.0,190.0
3,14,2800.0,2.0,2.0,3.0,400.0
4,15,300.0,2.0,2.0,4.0,90.0
...,...,...,...,...,...,...
1792,6246,1500.0,2.0,2.0,3.0,3200.0
1793,6252,1000.0,2.0,2.0,3.0,2900.0
1794,6253,600.0,2.0,2.0,3.0,3200.0
1795,6256,1000.0,2.0,2.0,3.0,2900.0


In [115]:
df4.info

<bound method DataFrame.info of         ID    LF07  EE01  EE02  EE03  incomeall
0        1  2500.0   2.0   2.0   3.0        4.0
1       11  4300.0   2.0   2.0   2.0      500.0
2       13  1700.0   1.0   2.0   3.0      190.0
3       14  2800.0   2.0   2.0   3.0      400.0
4       15   300.0   2.0   2.0   4.0       90.0
...    ...     ...   ...   ...   ...        ...
1792  6246  1500.0   2.0   2.0   3.0     3200.0
1793  6252  1000.0   2.0   2.0   3.0     2900.0
1794  6253   600.0   2.0   2.0   3.0     3200.0
1795  6256  1000.0   2.0   2.0   3.0     2900.0
1796  6265  3500.0   1.0   2.0   3.0     5500.0

[1792 rows x 6 columns]>